### Necessary Imports

In [ ]:
# Import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import torch.nn.functional as F

In [ ]:
import os
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download The Dataset & Define The Transforms

In [ ]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('/content/drive/My Drive/handwritten-digit-recognition-master/MNiST/', download=True, train=True, transform=transform)
valset = datasets.MNIST('/content/drive/My Drive/handwritten-digit-recognition-master/MNiST/', download=True, train=False, transform=transform)
testset = datasets.MNIST('/content/drive/My Drive/handwritten-digit-recognition-master/MNiST/', download=True, train=False,transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64*2, shuffle=True)
testloader= torch.utils.data.DataLoader(testset,batch_size=64*2, shuffle = True)

In [ ]:
from torch import nn

# Layer details for the neural network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
class MnistModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(input_size,hidden_sizes[0])
    self.activation1 = nn.ReLU()
    self.linear2 = nn.Linear(hidden_sizes[0],hidden_sizes[1])
    self.activation2 = nn.ReLU()
    self.linear3 = nn.Linear(hidden_sizes[1],output_size)
    self.activation3= nn.Softmax()
    self.dropout = nn.Dropout(0.2)

  def forward(self,xb):
    xb = xb.reshape(-1,784)
    x = self.linear1(xb)
    x = self.activation1(x)
    x = self.linear2(x)
    x = self.activation2(x)
    x = self.dropout(x)
    x = self.linear3(x)
    out = self.activation3(x)
    return out

  def training_step(self, batch):
    images, labels = batch 
    out = self(images)                  # Generate predictions
    loss = F.cross_entropy(out,labels) # Calculate loss
    return loss
   # acc = accuracy(out,labels)
   # return {'train_loss':loss.detach(), 'train_acc':acc.detach()}
  
  def train_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(out,labels)
    return {'train_loss':loss.detach(), 'train_acc':acc.detach()}
  
  def validation_step(self, batch):
    images, labels = batch 
    out = self(images)                    # Generate predictions
    loss = F.cross_entropy(out,labels)   # Calculate loss
    acc = accuracy(out, labels)           # Calculate accuracy
    return {'val_loss': loss.detach(), 'val_acc': acc.detach()}

  def train_epoch_end(self,outputs):
    batch_losses =[x['train_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accs = [x['train_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()
    return {'train_loss':epoch_loss.item(),'train_acc': epoch_acc.item()}
        
  def validation_epoch_end(self, outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
  def epoch_end_val(self, epoch, result):
    print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
  def epoch_end_train(self,epoch,result):
    print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}".format(epoch, result['train_loss'], result['train_acc']))

  

model = MnistModel()

In [ ]:

print(model.linear1)
print(model.linear2)
print(model.linear3)

Linear(in_features=784, out_features=128, bias=True)
Linear(in_features=128, out_features=64, bias=True)
Linear(in_features=64, out_features=10, bias=True)


In [ ]:
#print(model.linear3.weight)

In [ ]:
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))
  
def customize_one_hot_encoder(y):
  one_hot=F.one_hot(y,num_classes=output_size).float()
  one_hot[one_hot==0.0]=-1.0
  return one_hot

def Exponential_Loss(y_true,y_pred):
  y_true = customize_one_hot_encoder(y_true)
  Loss = torch.sum(torch.exp(-(torch.multiply(y_true,y_pred))))
  return Loss/len(y_true)


In [ ]:
def evaluate_validation(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def evaluate_training(model,train_loader):
  outputs = [model.train_step(batch) for batch in train_loader]
  return model.train_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    losses = []
    history_val =[]
    history_train= []
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result_train = evaluate_training(model,train_loader)
        model.epoch_end_train(epoch,result_train)
        history_train.append(result_train)
        # Validation phase
        losses.append(loss)
        result_val = evaluate_validation(model, val_loader)
        model.epoch_end_val(epoch, result_val)
        history_val.append(result_val)
        history = [history_train, history_val]
    return history

In [ ]:
#evaluate_validation(model,valloader)

In [ ]:
history = fit(11,0.1,model,trainloader,valloader)

Epoch [0], train_loss: 1.5031, train_acc: 0.9610
Epoch [0], val_loss: 1.5063, val_acc: 0.9577
Epoch [1], train_loss: 1.5058, train_acc: 0.9582
Epoch [1], val_loss: 1.5084, val_acc: 0.9554
Epoch [2], train_loss: 1.5258, train_acc: 0.9392
Epoch [2], val_loss: 1.5270, val_acc: 0.9369
Epoch [3], train_loss: 1.4971, train_acc: 0.9666
Epoch [3], val_loss: 1.5028, val_acc: 0.9605


In [ ]:
val_accuracies = [r['val_acc'] for r in history[1]]
train_accuracies = [s['train_acc'] for s in history[0]]
plt.plot(train_accuracies)
plt.plot(val_accuracies)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(["Train_Accuracy","Val_Accuracy"])
plt.title('Accuracy vs. No. of epochs');




NameError: ignored

In [ ]:
val_losses = [r['val_loss'] for r in history[1]]
train_losses = [s['train_loss'] for s in history[0]]
plt.plot(train_losses)
plt.plot(val_losses)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(["Train_loss", "Val_loss"])
plt.title('Loss vs No. of epochs');

In [ ]:
def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [ ]:
def correct_pred(img,label):
  if label == predict_image(img,model):
    str = "correct"    
  else:
    str = "incorrect"
  return str

In [ ]:
for i in range(1000):
  img, label = testset[i]
  #plt.imshow(img[], cmap='gray')
  output = correct_pred(img,label)
  print('Label:', label, ', Predicted:', predict_image(img, model) ,',Status:',output)

In [ ]:
def count_correct_pred(test_set):
  correct = 0
  incorrect = 0
  for i in range(1000):
    img, label = test_set[i]
    if label == predict_image(img,model):
      correct +=1
    else:
      incorrect +=1
  return correct,incorrect


In [ ]:
cor, incor = count_correct_pred(testset)

In [ ]:
print(cor)

In [ ]:
print(incor)

In [ ]:
print(cor+incor)